In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import os
import sys
import time
import numpy as np
import math
import json
from tqdm import tqdm_notebook as tqdm
import pathlib
from pathlib import Path
import time

import matplotlib.pyplot as plt 
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
from torch_geometric.nn import (NNConv, GMMConv, GraphConv, Set2Set)
from torch_geometric.nn import (SplineConv, graclus, max_pool, max_pool_x, global_mean_pool)

#from neuralnet_pytorch.metrics import chamfer_loss

import trimesh

from visualization_utils import plot_mesh_3d

import deep_sdf
import deep_sdf.workspace as ws
from models import *
from datasets import *
from custom_utils import *

from sklearn.neighbors import KDTree

In [2]:
def computeAvgTransform():
    objects = list()
    for (dirpath, dirnames, filenames) in os.walk("/cvlabdata2/home/artem/Data/cars_remeshed_dsdf/transforms/"):
        objects += [os.path.join(dirpath, file) for file in filenames if file[-4:] == '.npy']
    
    matricies = []
    for obj in objects:
        matricies.append(np.load(obj))
    
    return np.mean(np.array(matricies), axis=0)

AvgTransform = computeAvgTransform()

def transformPoints(points, matrix):
    matrix = torch.cuda.FloatTensor(matrix)
    column = torch.zeros((len(points), 1), device="cuda:0") + 1
    stacked = torch.cat([points, column], dim=1)
    transformed = torch.matmul(matrix, stacked.T).T[:, :3]
    return transformed

def make_mesh_from_points(points, ply_mesh):
    transformed_points = transformPoints(points, AvgTransform)
    
    edges = trimesh.geometry.faces_to_edges(ply_mesh['face']['vertex_indices'])
    np_points = transformed_points.cpu().detach().numpy()
    edge_attr = [np_points[a] - np_points[b] for a, b in edges]

    data = torch_geometric.data.Data(x   = transformed_points, 
                                     pos = transformed_points, 
                                     face = torch.tensor(ply_mesh['face']['vertex_indices'], 
                                                         dtype=torch.long).to('cuda:0').t(),
                                     edge_attr=torch.tensor(edge_attr, dtype=torch.float).to('cuda:0'),
                                     edge_index=torch.tensor(edges, dtype=torch.long).t().contiguous().to('cuda:0'))
    return data


def compute_volume(vertices_in, faces_in):

    bs, nv = vertices_in.shape[:2]
    bs, nf = faces_in.shape[:2]
    device = vertices_in.device
    faces = faces_in + (torch.arange(bs, dtype=torch.int32).to(device) * nv)[:, None, None]
    vertices = vertices_in #.reshape((bs * nv / 2, 3))
    # pytorch only supports long and byte tensors for indexing
    face_coordinates = vertices[faces.long()].squeeze(0)

    x1y2z3 = face_coordinates[:,0,0]*face_coordinates[:,1,1]*face_coordinates[:,2,2]
    x2y1z3 = face_coordinates[:,1,0]*face_coordinates[:,0,1]*face_coordinates[:,2,2]
    x3y1z2 = face_coordinates[:,2,0]*face_coordinates[:,0,1]*face_coordinates[:,1,2]
    x1y3z2 = face_coordinates[:,0,0]*face_coordinates[:,2,1]*face_coordinates[:,1,2]
    x2y3z1 = face_coordinates[:,1,0]*face_coordinates[:,2,1]*face_coordinates[:,0,2]
    x3y2z1 = face_coordinates[:,2,0]*face_coordinates[:,1,1]*face_coordinates[:,0,2]

    V = x1y2z3 - x2y1z3 + x3y1z2 - x1y3z2 + x2y3z1 - x3y2z1
    return 1.0/6.0*torch.abs(torch.sum(V))


def boundsLoss(points, box=[(-1, 1, 0)]):
    loss = 0
    for l, r, i in box:
        loss +=  torch.mean(F.relu(-points[:, i] + l))  \
               + torch.mean(F.relu( points[:, i] - r))
    return loss

def innerBoundsLoss(points, r=1, center=(0, 0, 0)):
    radiuses = torch.sum( (points - torch.Tensor(center).to('cuda:0')) ** 2 , dim=1)
    return torch.sum(F.relu(r - radiuses))

def calculate_loss(mesh, local_preds, axis=0, constraint_rad=0.1):
    loss =  (1 - axis) * compute_lift_faces_diff(mesh, local_preds, axis=0) + \
                  axis * compute_lift_faces_diff(mesh, local_preds, axis=1)
    
    loss += boundsLoss(mesh.x, box=[(-0.6, 0.6, 0)])
    loss += innerBoundsLoss(mesh.x, r=constraint_rad**2, center=(-0.05, 0.05, 0))  \
          + innerBoundsLoss(mesh.x, r=(constraint_rad / 2)**2, center=(0.3, 0, 0))
    

    return loss

In [3]:
!pwd

/cvlabdata2/home/artem/DeepSDF


In [4]:
# data_instance= make_data_instance_from_stl(
#                '/cvlabdata2/home/artem/Data/cars_remeshed_dsdf/outputs/fld/0003_0015.fld') # /cvlabdata2/home/artem/Data/cars_refined/simulated/fld/0002_0005.fld
device = "cuda:0"

# modelOld = SplineCNN8(3, batchnorm1=False)
# modelOld.load_state_dict(torch.load("Expirements/SplineCNN8.nn"))
# modelOld = modelOld.to(device)

model = SplineCNN8Residuals(3)
model.load_state_dict(torch.load("Expirements/Networks15/normilized_full_latest.nn"))
model = model.to(device)
model = model.eval()

In [5]:
experiment_directory = "/cvlabdata2/home/artem/DeepSDF/examples/cars_cleared/"
checkpoint = "latest"
decoder = load_model(experiment_directory, checkpoint).to('cuda:0')

latent_vectors = ws.load_latent_vectors(experiment_directory, checkpoint)
latent_size = latent_vectors[0].size()[0]
print(f"{len(latent_vectors)} of latent vectors, each {latent_size} long")

1205 of latent vectors, each 1 long


# Vertex-vise Optimization

In [6]:
def optimize_shape_by_vertex(model, inp, num_iters=30, save_to_dir='Expirements/Optimization',
                   lr=0.05, decreased_by=2, adjust_lr_every=10, verbose=None, constraint_rad=0.1, axis=0):

    def adjust_learning_rate(
        initial_lr, optimizer, num_iterations, decreased_by, adjust_lr_every
    ):
        lr = initial_lr * ((1 / decreased_by) ** (num_iterations // adjust_lr_every))
        for param_group in optimizer.param_groups:
            param_group["lr"] = lr
            
        return lr
    
    if not os.path.exists(os.path.join(save_to_dir, 'meshes')):
        os.makedirs(os.path.join(save_to_dir, 'meshes'))
    if not os.path.exists(os.path.join(save_to_dir, 'predictions')):
        os.makedirs(os.path.join(save_to_dir, 'predictions'))

    model.eval()
    data_instance = inp.clone()
    data_instance.x.requires_grad = True
    optimizer = torch.optim.SGD([data_instance.x], lr=lr)

    meshes = []
    lifts = []
    lr_plot = []

    for i in range(num_iters):
        cur_lr = adjust_learning_rate(lr, optimizer, i, decreased_by, adjust_lr_every)

        save_path = os.path.join(save_to_dir, 'meshes/' + str(i).zfill(5) + ".ply")
        preds_save_path = os.path.join(save_to_dir, 'predictions/' + str(i).zfill(5) + ".npy")

        optimizer.zero_grad()
        local_preds = model(data_instance)
#             # We get input as CFD mesh, but the constraints are specified in DeepSDF
#             # We save in CFD mesh, so no need to transform before simulation
#             points_in_deepSDF = transformPoints(data_instance.x, np.linalg.inv(AvgTransform))

        #print(points_in_deepSDF)
        loss = calculate_loss(data_instance, local_preds,
                              axis=axis, constraint_rad=constraint_rad)

        loss.backward()
        #print('Avg grad: ', torch.mean(torch.sum(data_instance.x.grad ** 2, axis=1)))

        tri_mesh = get_trimesh_from_torch_geo_with_colors(data_instance, local_preds)
        tri_mesh.export(save_path)
        np.save(preds_save_path, local_preds.cpu().detach().numpy())

#             data_instance.x.grad[torch.isnan(data_instance.x.grad)] = 0
        optimizer.step()

        if verbose is not None and i % verbose == 0:
            print('Iter ', i, 'Loss: ', loss.detach().cpu().numpy(), ' LR: ', cur_lr)
                #plot_points_from_torch(points)

        lifts.append(loss.detach().cpu().numpy())
        lr_plot.append(cur_lr)
        np.save(os.path.join(save_to_dir, "loss_plot.npy"), lifts)
        np.save(os.path.join(save_to_dir, "lr_plot.npy"), lr_plot)

In [7]:
# optimize_shape_by_vertex(model, MESH_TO_OPTIMIZE, verbose=1, save_to_dir='Expirements/OptimizationPaper/NewGenVertexDragTight', 
#                          lr=0.002, adjust_lr_every=20)

# BB Optimization

In [8]:
def optimize_shape_by_scaling(model, inp, num_iters=100, save_to_dir='Expirements/Optimization',
                   lr=0.05, decreased_by=2, adjust_lr_every=10, verbose=None, constraint_rad=0.1, axis=0):

    def transform(data, scale):
        answ = data.clone()
        
        answ.x =  data.x * scale[1]
        answ.pos = answ.x
       # answ.edge_attr = answ.x[ answ.edge_index[0] ] - answ.x[answ.edge_index[1]] 
        return answ
    
    def adjust_learning_rate(
        initial_lr, optimizer, num_iterations, decreased_by, adjust_lr_every
    ):
        lr = initial_lr * ((1 / decreased_by) ** (num_iterations // adjust_lr_every))
        for param_group in optimizer.param_groups:
            param_group["lr"] = lr
            
        return lr

    if not os.path.exists(os.path.join(save_to_dir, 'meshes')):
        os.makedirs(os.path.join(save_to_dir, 'meshes'))
    if not os.path.exists(os.path.join(save_to_dir, 'predictions')):
        os.makedirs(os.path.join(save_to_dir, 'predictions'))

    model.eval()
    data_instance = inp.clone()
    data_instance.x.requires_grad = True
    scale = torch.Tensor([[0.0, 0.0, 0.0], [1.0, 1.0, 1.0]]).to("cuda:0")
    scale.requires_grad = True

    optimizer = torch.optim.SGD([scale], lr=lr)

    meshes = []
    lifts = []
    lr_plot = []

    for i in range(num_iters):
        cur_rl = adjust_learning_rate(lr, optimizer, i, decreased_by, adjust_lr_every)
        save_path = os.path.join(save_to_dir, 'meshes/' + str(i).zfill(5) + ".ply")
        preds_save_path = os.path.join(save_to_dir, 'predictions/' + str(i).zfill(5) + ".npy")

        optimizer.zero_grad()
        transformed_mesh = transform(data_instance, scale)

        # Restriction
        # min_dist = compute_min_distance(transformed_mesh)
        # scale_clapped = scale * min(1, GLOBAL_MIN_DIST / min_dist)
        #transformed_mesh = transform(data_instance, scale)

        local_preds = model(transformed_mesh)
#             points_in_deepSDF = transformPoints(transformed_mesh.x, np.linalg.inv(AvgTransform))
        loss = calculate_loss(transformed_mesh, local_preds, axis=axis, constraint_rad=constraint_rad)

        loss.backward()
        optimizer.step()

        if verbose is not None and i % verbose == 0:
            print('Iter ', i, 'Loss: ', loss.detach().cpu().numpy(), ' LR: ', cur_rl)
                #plot_points_from_torch(points)

        tri_mesh = get_trimesh_from_torch_geo_with_colors(transformed_mesh, local_preds)
        tri_mesh.export(save_path)
        np.save(preds_save_path, local_preds.cpu().detach().numpy())

        lifts.append(loss.detach().cpu().numpy())
        lr_plot.append(cur_rl)

        np.save(os.path.join(save_to_dir, "loss_plot.npy"), lifts)
        np.save(os.path.join(save_to_dir, "lr_plot.npy"), lr_plot)

In [9]:
# optimize_shape_by_scaling(model, MESH_TO_OPTIMIZE, verbose=1, save_to_dir='Expirements/OptimizationPaper/BboxDragDiff', lr=0.05)

# Free Foam Optimization

In [10]:
def optimize_shape_as_pierre(model, inp, num_iters=100, save_to_dir='Expirements/Optimization',
                             lr=0.05, decreased_by=2, adjust_lr_every=10, verbose=None, 
                             constraint_rad=0.1, axis=0, regularization=1):

    def transform(data, scale):
        answ = data.clone() #scale[0]
        answ.x = \
            data.x * (scale[1] + scale[2] * data.x + \
                      scale[3] * torch.cos(data.x[:, (2, 0, 1)] * 2 * math.pi) + \
                      scale[4] * torch.cos(data.x[:, (1, 2, 0)] * 2 * math.pi) + \
                      scale[5] * torch.sin(data.x[:, (2, 0, 1)] * 2 * math.pi) ** 2 + \
                      scale[5] * torch.sin(data.x[:, (1, 2, 0)] * 2 * math.pi) ** 2)
        answ.pos = answ.x
#         answ.edge_attr = answ.x[ answ.edge_index[0] ] - answ.x[answ.edge_index[1]] 
        return answ
    
    def adjust_learning_rate(
        initial_lr, optimizer, num_iterations, decreased_by, adjust_lr_every
    ):
        lr = initial_lr * ((1 / decreased_by) ** (num_iterations // adjust_lr_every))
        for param_group in optimizer.param_groups:
            param_group["lr"] = lr
            
        return lr

    if not os.path.exists(os.path.join(save_to_dir, 'meshes')):
        os.makedirs(os.path.join(save_to_dir, 'meshes'))
    if not os.path.exists(os.path.join(save_to_dir, 'predictions')):
        os.makedirs(os.path.join(save_to_dir, 'predictions'))
    
    model.eval()
    data_instance = inp.clone()
    data_instance.x.requires_grad = True
    scale = torch.Tensor([[0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
                          [0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
                          [0.0, 1.0, 0.0, 0.0, 0.0, 0.0]]).to("cuda:0").t()
    scale.requires_grad = True

    optimizer = torch.optim.SGD([scale], lr=lr)

    meshes = []
    lifts = []
    lr_plot = []

    for i in range(num_iters):
        cur_rl = adjust_learning_rate(lr, optimizer, i, decreased_by, adjust_lr_every)
        save_path = os.path.join(save_to_dir, 'meshes/' + str(i).zfill(5) + ".ply")
        preds_save_path = os.path.join(save_to_dir, 'predictions/' + str(i).zfill(5) + ".npy")

        optimizer.zero_grad()
        transformed_mesh = transform(data_instance, scale)

        local_preds = model(transformed_mesh)
#             points_in_deepSDF = transformPoints(transformed_mesh.x, np.linalg.inv(AvgTransform))
        loss = calculate_loss(transformed_mesh, local_preds, axis=axis, constraint_rad=constraint_rad)

        loss.backward()
        l2_norm = regularization * torch.mean(scale[2:] ** 2)
        l2_norm.backward()
#         print("Grad: ", scale.grad)
        optimizer.step()

        if verbose is not None and i % verbose == 0:
            print('Iter ', i, 'Loss: ', loss.detach().cpu().numpy(), ' LR: ', cur_rl)
                #plot_points_from_torch(points)

        tri_mesh = get_trimesh_from_torch_geo_with_colors(transformed_mesh, local_preds)
        tri_mesh.export(save_path)
        np.save(preds_save_path, local_preds.cpu().detach().numpy())

        lifts.append(loss.detach().cpu().numpy())
        lr_plot.append(cur_rl)

        np.save(os.path.join(save_to_dir, "loss_plot.npy"), lifts)
        np.save(os.path.join(save_to_dir, "lr_plot.npy"), lr_plot)


In [11]:
# i=175
# optimize_shape_as_pierre(model, MESH_TO_OPTIMIZE, verbose=verbose, constraint_rad=0.1, num_iters=num_iters,
#                                  save_to_dir='Expirements/OptimizationPaper/AfterMeeting/FreeformDrag/' + str(i), 
#                                  lr=0.05, adjust_lr_every=20, axis=a, regularization=0.5)



# Free Foam Furrier Optimization

In [12]:
def optimize_shape_as_pierre_furrier(model, inp, num_params=88, num_iters=100, save_to_dir='Expirements/Optimization',
                   lr=0.05, decreased_by=2, adjust_lr_every=10, verbose=None, 
                                     constraint_rad=0.1, axis=0, regularization=1):

    def transform(data, scale):
        answ = data.clone()
        answ.x = \
            data.x * (scale[1] + scale[2] * data.x)
        for k, idx in enumerate(range(3, num_params - 3, 4)):
            answ.x += data.x * (scale[idx + 0] * torch.cos((k + 1) * data.x[:, (2, 0, 1)] * 2 * math.pi) + \
                                scale[idx + 1] * torch.cos((k + 1) * data.x[:, (1, 2, 0)] * 2 * math.pi) + \
                                scale[idx + 2] * torch.sin((k + 1) * data.x[:, (2, 0, 1)] * 2 * math.pi) + \
                                scale[idx + 3] * torch.sin((k + 1) * data.x[:, (1, 2, 0)] * 2 * math.pi))
        answ.pos = answ.x
#         answ.edge_attr = answ.x[ answ.edge_index[0] ] - answ.x[answ.edge_index[1]] 
        return answ
    
    def adjust_learning_rate(
        initial_lr, optimizer, num_iterations, decreased_by, adjust_lr_every
    ):
        lr = initial_lr * ((1 / decreased_by) ** (num_iterations // adjust_lr_every))
        for param_group in optimizer.param_groups:
            param_group["lr"] = lr
            
        return lr

    if not os.path.exists(os.path.join(save_to_dir, 'meshes')):
        os.makedirs(os.path.join(save_to_dir, 'meshes'))
    if not os.path.exists(os.path.join(save_to_dir, 'predictions')):
        os.makedirs(os.path.join(save_to_dir, 'predictions'))

    model.eval()
    data_instance = inp.clone()
    data_instance.x.requires_grad = True
    scale = torch.zeros((3, num_params), dtype=torch.float).to("cuda:0").t()
    scale[1] = 1
    scale.requires_grad = True

    optimizer = torch.optim.SGD([scale], lr=lr)

    meshes = []
    lifts = []
    lr_plot = []

    for i in range(num_iters):
        cur_rl = adjust_learning_rate(lr, optimizer, i, decreased_by, adjust_lr_every)
        save_path = os.path.join(save_to_dir, 'meshes/' + str(i).zfill(5) + ".ply")
        preds_save_path = os.path.join(save_to_dir, 'predictions/' + str(i).zfill(5) + ".npy")

        optimizer.zero_grad()
        transformed_mesh = transform(data_instance, scale)

        local_preds = model(transformed_mesh)
#             points_in_deepSDF = transformPoints(transformed_mesh.x, np.linalg.inv(AvgTransform))
        loss = calculate_loss(transformed_mesh, local_preds, axis=axis, constraint_rad=constraint_rad) 

        loss.backward()
        
        # Regularization
        l2_norm = regularization * torch.mean(scale[2:] ** 2)
        l2_norm.backward()
        
        optimizer.step()

        if verbose is not None and i % verbose == 0:
            print('Iter ', i, 'Loss: ', loss.detach().cpu().numpy(), ' LR: ', cur_rl)
                #plot_points_from_torch(points)

        tri_mesh = get_trimesh_from_torch_geo_with_colors(transformed_mesh, local_preds)
        tri_mesh.export(save_path)
        np.save(preds_save_path, local_preds.cpu().detach().numpy())

        lifts.append(loss.detach().cpu().numpy())
        lr_plot.append(cur_rl)

        np.save(os.path.join(save_to_dir, "loss_plot.npy"), lifts)
        np.save(os.path.join(save_to_dir, "lr_plot.npy"), lr_plot)

In [13]:
# optimize_shape_as_pierre_furrier(model, MESH_TO_OPTIMIZE, verbose=1, 
#                                  save_to_dir='Expirements/OptimizationPaper/FurrierDragDiff', lr=0.005)

# AtlasNet Optimization

In [14]:
# modelAN = SplineCNN8Residuals(3)
# modelAN.load_state_dict(torch.load("Expirements/Networks15/AtlasCNN8ResNormMapping_latest.nn"))
# modelAN = modelAN.to(device)
# modelAN = modelAN.eval()

# sys.path.append("/cvlabdata2/home/artem/DeepSDF/atlasnet_retrained") # '/cvlabdata2/home/artem/DeepSDF/atlasnet/'

# import dependencies.dataset_shapenet as dataset_shapenet
# from dependencies.model import EncoderDecoder
# import dependencies.argument_parser as argument_parser

# from easydict import EasyDict
# import pdb
# import pymesh

In [15]:
def optimize_shape_atlasNet(model, save_to_dir, latent_idx, 
                            num_iters=100, lr=5e-3, verbose=None, axis=0, constraint_rad=0.1):
    
    def make_mesh_from_mesh_atlas(points, faces):
    
        transformed_points = points#transformPoints(points, AvgTransform)

        edges = trimesh.geometry.faces_to_edges(faces)
        np_points = transformed_points.clone().cpu().detach().numpy()
        edge_attr = [np_points[a] - np_points[b] for a, b in edges]

        data = torch_geometric.data.Data(x  = transformed_points, 
                                         pos= transformed_points, 
                                         face = torch.tensor(faces, 
                                                         dtype=torch.long).to('cuda:0').t(),
                                         edge_attr=torch.tensor(edge_attr, dtype=torch.float).to('cuda:0'),
                                         edge_index=torch.tensor(edges, dtype=torch.long).to('cuda:0').t().contiguous())
        return data


    sys.argv = ['foo']
    opt = argument_parser.parser()
    torch.cuda.set_device(opt.multi_gpu[0])


    """
    load pre-trained model
    """
    OUT_PATH = "/cvlabdata2/home/artem/DeepSDF/atlasnet_retrained/optimizations/"
    MODEL_PATH = "/cvlabdata2/home/artem/DeepSDF/atlasnet_retrained/trained/network.pth"


    if torch.cuda.is_available():
        opt.device = torch.device(f"cuda:{opt.multi_gpu[0]}")
    else:
        # Run on CPU
        opt.device = torch.device(f"cpu")

    network = EncoderDecoder(opt)
    network = nn.DataParallel(network, device_ids=opt.multi_gpu)
    # load weights
    network.load_state_dict(torch.load(MODEL_PATH, map_location='cuda:0'))
    # finally keep only decoder
    decoder = nn.DataParallel(network.module.decoder)
    # and put in eval mode
    decoder.eval()

    """
    load pre-trained model
    """

    # init
    latent_filename = os.path.join(
        "/cvlabdata2/home/artem/DeepSDF/atlasnet_retrained/reconstructions/codes", latent_idx + ".pth"
    )
    latent = torch.load(latent_filename).squeeze(0).clone().detach().requires_grad_(True)
    latent.requires_grad = True

    optimizer = torch.optim.Adam([latent], lr=lr)
    
    '''
    Dealing with pathes
    '''
    
    if not os.path.exists(os.path.join(save_to_dir, 'meshes')):
        os.makedirs(os.path.join(save_to_dir, 'meshes'))
    if not os.path.exists(os.path.join(save_to_dir, 'predictions')):
        os.makedirs(os.path.join(save_to_dir, 'predictions'))
        
    loss_plot = []

    for i in range(num_iters):
        save_path = os.path.join(save_to_dir, 'meshes/' + str(i).zfill(5) + ".ply")
        preds_save_path = os.path.join(save_to_dir, 'predictions/' + str(i).zfill(5) + ".npy")
            
        optimizer.zero_grad()
        
        # create mesh
        verts, faces = decoder.module.generate_mesh(latent.unsqueeze(0))
        
        mesh = make_mesh_from_mesh_atlas(verts, faces)
        # predict
        local_preds = model(mesh)
        loss = calculate_loss(mesh, local_preds, axis=axis, constraint_rad=constraint_rad)
        
        loss.backward()
        optimizer.step()
        
        loss_plot.append(loss.detach().cpu().numpy())
        
        if verbose is not None and i % verbose == 0:
             print("Loss at iter", i, ":", loss_plot[-1])

        np.save(preds_save_path, local_preds.cpu().detach().numpy())
        np.save(os.path.join(save_to_dir, "loss_plot.npy"), loss_plot)
        mesh_save = get_trimesh_from_torch_geo_with_colors(mesh, local_preds)
        mesh_save.export(save_path)

    print("Done.")

In [16]:
def calculate_loss(mesh, local_preds, axis=0, constraint_rad=0.1):
    loss =  (1 - axis) * compute_lift_faces_diff(mesh, local_preds, axis=0) + \
                  axis * compute_lift_faces_diff(mesh, local_preds, axis=1)
    
#     loss += boundsLoss(mesh.x, box=[(-0.6, 0.6, 0)])
#     loss += innerBoundsLoss(mesh.x, r=constraint_rad**2, center=(-0.05, 0.05, 0))  \
#           + innerBoundsLoss(mesh.x, r=(constraint_rad / 2)**2, center=(0.3, 0, 0))

    #loss += compute_volume(mesh.x[None, :, :], mesh.face.t()[None, :, :])
    return loss

# optimize_shape_atlasNet(modelAN,'Expirements/OptimizationPaper/AfterMeeting/AtlasNetDrag/22', latent_idx='0022',
#                         lr=5e-3, verbose=1,  num_iters=30)
# optimize_shape_atlasNet(modelAN,'Expirements/OptimizationPaper/FairLoss/23', latent_idx='0023',
#                         lr=5e-3, verbose=1,  num_iters=30)
# optimize_shape_atlasNet(modelAN,'Expirements/OptimizationPaper/FairLoss/24', latent_idx='0024',
#                         lr=5e-3, verbose=1,  num_iters=30)
# optimize_shape_atlasNet(modelAN,'Expirements/OptimizationPaper/FairLoss/25', latent_idx='0025',
#                         lr=5e-3, verbose=1,  num_iters=30)
# optimize_shape_atlasNet(modelAN,'Expirements/OptimizationPaper/FairLoss/26', latent_idx='0026',
#                         lr=5e-3, verbose=1,  num_iters=30)

# DeepSDF Optimization

In [17]:
def method4_to_arbitatry_loss(points, ply_mesh, model, constraint_rad=0.1, axis=0):

    initial_dir = points.grad.clone()
    points.grad.data.zero_()

    mesh = make_mesh_from_points(points, ply_mesh)
    #signs = compute_signs_for_loss(mesh, transformPoints(normals, AvgTransform))
    local_preds = model(mesh)
    loss = calculate_loss(mesh, local_preds, axis=axis, constraint_rad=constraint_rad)
    loss.backward()

    sign = [-p1.dot(p2) for p1, p2 in zip(initial_dir, points.grad)]
    
    return sign, loss, local_preds, mesh

def optimize_shape_deepSDF(decoder, latent, ref_latent, initial_points=None, num_points=None, 
                           num_iters=100, point_iters=100,  punch_lr_at_reindex_by=1, num_neignours_constr=10,
                           reindex_latent_each=50, reindex_num_iterations=500, reindex_num_samples=100,
                           lr=0.2, decreased_by=2, adjust_lr_every=10, alpha_penalty=0.05,
                           multiplier_func=method4_to_arbitatry_loss, verbose=None, save_to_dir=None, N=256):

    def adjust_learning_rate(
        initial_lr, optimizer, num_iterations, decreased_by, adjust_lr_every
    ):
        lr = initial_lr * ((1 / decreased_by) ** (num_iterations // adjust_lr_every)) \
                        * ((punch_lr_at_reindex_by) ** (num_iterations // reindex_latent_each))
        for param_group in optimizer.param_groups:
            param_group["lr"] = lr
            
        return lr
    
    if not os.path.exists(os.path.join(save_to_dir, 'meshes')):
        os.makedirs(os.path.join(save_to_dir, 'meshes'))
    if not os.path.exists(os.path.join(save_to_dir, 'predictions')):
        os.makedirs(os.path.join(save_to_dir, 'predictions'))

    decoder.eval()
    latent = latent.clone()
    latent.requires_grad = True
    optimizer = torch.optim.SGD([latent], lr=lr)

    loss_plot = []
    latent_dist = []
    lr_plot = []

    if initial_points is not None:
        points = initial_points.clone()
    else:
        points = get_points_from_latent(decoder, latent, N=N, point_num=num_points)

    for i in range(num_iters):

        time_start = time.time()

        save_path = os.path.join(save_to_dir, 'meshes/' + str(i).zfill(5) + ".ply")
        preds_save_path = os.path.join(save_to_dir, 'predictions/' + str(i).zfill(5) + ".npy")

#             if i > 0 and i == reindex_latent_each:
#                 new_latent = get_latent_from_mesh(decoder, latent_size=latent.size()[1], 
#                                                   num_iterations=reindex_num_iterations, 
#                                                   num_samples=reindex_num_samples)
#                 latent = torch.Tensor(new_latent.cpu().detach().numpy()).cuda()
#                 latent.requires_grad = True
#                 optimizer = torch.optim.Adam([latent], lr=lr)

        cur_rl = adjust_learning_rate(lr, optimizer, i, decreased_by, adjust_lr_every)

        with torch.no_grad():
            ply_mesh = create_mesh( decoder,
                                    latent,
                                    N=N,
                                    max_batch=int(2 ** 18),
                                    offset=None,
                                    scale=None)

        points = torch.cuda.FloatTensor(np.hstack(( ply_mesh['vertex']['x'][:, None], 
                                                    ply_mesh['vertex']['y'][:, None], 
                                                    ply_mesh['vertex']['z'][:, None])))

        points.requires_grad = True

        sdf_value = deep_sdf.utils.decode_sdf(decoder, latent, points)
        sdf_value.backward(torch.ones([len(points), 1], dtype=torch.float32).cuda())

        mults, loss_value, preds, transformed_mesh = multiplier_func(points, ply_mesh)         
        multipliers = torch.cuda.FloatTensor(mults)

        optimizer.zero_grad()
        sdf_value = torch.squeeze(deep_sdf.utils.decode_sdf(decoder, latent, points))

        final_loss = torch.sum(sdf_value * multipliers)
        final_loss.backward()
#             print( "Latent size: ", torch.sum(latent ** 2) )
#             print( "Latent grad: ", torch.sum(latent.grad ** 2) )

#           sdf_value.backward(multipliers)


        # Soft-constraints
        distances, indeces = LATENT_KD_TREE.query(latent.cpu().detach(), k=num_neignours_constr)
        penalty = torch.mean(
                    torch.stack([torch.sum( 
                                    (latent - latent_vectors[indeces[0][i]]) ** 2
                                 )
                                 for i in range(len(indeces[0]))]
                               )
                    )
        apenalty = penalty * alpha_penalty
        apenalty.backward()
        #print("Latent grad penalized: ", torch.sum(latent.grad ** 2))

        optimizer.step()

        # Hard-constraints
#             if (torch.sum(latent ** 2) > 1.2):
#                 latent *= 1.2 / torch.sum(latent ** 2)

#             loss_value, preds = loss_func(points, ply_mesh)            

        tri_mesh = get_trimesh_from_torch_geo_with_colors(transformed_mesh, preds)
        tri_mesh.export(save_path)
        np.save(preds_save_path, preds.cpu().detach().numpy())

        if save_to_dir is not None:
            plot_points_from_torch

        loss_plot.append(loss_value.cpu().detach().numpy())
        latent_dist.append(torch.sum((latent - ref_latent) ** 2 ).cpu().detach().numpy() )
        lr_plot.append(penalty)

        time_end = time.time()

        if verbose is not None and i % verbose == 0:
            print('Iter ', i, 'Loss: ', loss_value.detach().cpu().numpy(), ' LD: ', lr_plot[-1])

        np.save(os.path.join(save_to_dir, "loss_plot.npy"), loss_plot)
        np.save(os.path.join(save_to_dir, "latent_dist.npy"), latent_dist)
        np.save(os.path.join(save_to_dir, "lr_plot.npy"), lr_plot)
        np.save(os.path.join(save_to_dir, "latent%d.npy" % i), latent.cpu().detach().numpy())


def make_full_transformation(initial_latent, ref_latent, experiment_name, 
                             decoder, model, alpha_penalty=0.05, constraint_rad=0.1, axis=0, **kwargs):
    '''
    kwargs:
        num_iters=1000, 
        adjust_lr_every=10, 
        decreased_by=1.2,
        lr=0.005,
        
        reindex_latent_each=100,
        punch_lr_at_reindex_by=1,
        reindex_num_iterations=500, 
        reindex_num_samples=100,
        
        verbose=10,
    '''

    #ref_points = get_points_from_latent(decoder, ref_latent, N=128)

    save_to_dir = experiment_name
    if not os.path.exists(save_to_dir):
        os.makedirs(save_to_dir)

    #np.save(os.path.join(save_to_dir, "target_verts.npy"), ref_points)

    optimize_shape_deepSDF(decoder, initial_latent, ref_latent, initial_points=None,
                                           alpha_penalty=alpha_penalty,
                                           num_points=None, point_iters=2,
                                           multiplier_func=lambda x, y: 
                                               method4_to_arbitatry_loss(x, y, model, 
                                                                         constraint_rad=constraint_rad, 
                                                                         axis=axis),
                                           save_to_dir=save_to_dir, **kwargs)
    

## Run Everything

In [18]:
def compute_volume(vertices_in, faces_in):
    bs, nv = vertices_in.shape[:2]
    bs, nf = faces_in.shape[:2]
    device = vertices_in.device
    faces = faces_in + (torch.arange(bs, dtype=torch.int32).to(device) * nv)[:, None, None]
    vertices = vertices_in.reshape((bs * nv, 3))
    # pytorch only supports long and byte tensors for indexing
    face_coordinates = vertices[faces.long()].squeeze(0)

    x1y2z3 = face_coordinates[:,0,0]*face_coordinates[:,1,1]*face_coordinates[:,2,2]
    x2y1z3 = face_coordinates[:,1,0]*face_coordinates[:,0,1]*face_coordinates[:,2,2]
    x3y1z2 = face_coordinates[:,2,0]*face_coordinates[:,0,1]*face_coordinates[:,1,2]
    x1y3z2 = face_coordinates[:,0,0]*face_coordinates[:,2,1]*face_coordinates[:,1,2]
    x2y3z1 = face_coordinates[:,1,0]*face_coordinates[:,2,1]*face_coordinates[:,0,2]
    x3y2z1 = face_coordinates[:,2,0]*face_coordinates[:,1,1]*face_coordinates[:,0,2]

    V = x1y2z3 - x2y1z3 + x3y1z2 - x1y3z2 + x2y3z1 - x3y2z1
    ans = 1.0/6.0*torch.abs(torch.sum(V))
    print(ans)
    return ans

def calculate_loss(mesh, local_preds, axis=0, constraint_rad=0.1):
    loss =  -(1 - axis) * compute_lift_faces_diff(mesh, local_preds, axis=0) + \
                  axis * compute_lift_faces_diff(mesh, local_preds, axis=1)
    
    loss += 10 * boundsLoss(mesh.x, box=[(-0.6, 0.6, 0)])
    loss += innerBoundsLoss(mesh.x, r=(constraint_rad / 2)**2, center=(-0.1, 0.05, 0))
    loss += innerBoundsLoss(mesh.x, r=(constraint_rad / 2)**2, center=(-0.3, 0, 0))

    #loss += compute_volume(mesh.x[None, :, :], mesh.face.t()[None, :, :])
    return loss

In [21]:
experiment_dir = 'AfterMeeting'
num_iters = 40
N = 256
verbose = 30

spoilers_detach = [246]
spoilers_sportives = [64, 95, 118]
nice_shapes = [175, 185, 113, 69, 61, 49, 64]

nice_shapes2 = [61, 69, 113, 185]
for i in [535, 69, 32, 162, 61]:
    LATENT_TO_OPTIMIZE = latent_vectors[i]
    LATENT_KD_TREE = KDTree(np.array([lv.cpu().detach().numpy()[0] for lv in latent_vectors]))

    with torch.no_grad():
        ply_mesh = create_mesh( decoder,
                                LATENT_TO_OPTIMIZE,
                                N=N,
                                max_batch=int(2 ** 18),
                                offset=None,
                                scale=None)

    points = torch.cuda.FloatTensor(np.hstack(( ply_mesh['vertex']['x'][:, None], 
                                            ply_mesh['vertex']['y'][:, None], 
                                            ply_mesh['vertex']['z'][:, None])))
    normals = torch.cuda.FloatTensor(np.hstack(( ply_mesh['normals']['x'][:, None], 
                                                 ply_mesh['normals']['y'][:, None], 
                                                 ply_mesh['normals']['z'][:, None])))

    MESH_TO_OPTIMIZE = make_mesh_from_points(points, ply_mesh)
    #SAVED_SIGNS = compute_signs_for_loss(MESH_TO_OPTIMIZE, transformPoints(normals, AvgTransform))

    print("Starting shape ", i)
    
    for a, name in zip([0], ['Drag']):
#         DeepSDF
#         make_full_transformation(LATENT_TO_OPTIMIZE.detach(), LATENT_TO_OPTIMIZE.clone(), 
#                                     'Expirements/OptimizationPaper/' + experiment_dir + 
#                                          '/DeepSDF' + name + 'constr/' + str(i), decoder, model,
#                          alpha_penalty=0.2, axis=a,
#                          num_iters=num_iters,
#                          adjust_lr_every=20,
#                          decreased_by=1.1, 
#                          lr=0.2,
#                          verbose=None,
#                          N=N,
#                          num_neignours_constr=10,
#                          reindex_latent_each=10000,
#                          punch_lr_at_reindex_by=1,
#                          reindex_num_iterations=500,
#                          reindex_num_samples=100)

#         FreeFormFurrier
#         optimize_shape_as_pierre_furrier(model, MESH_TO_OPTIMIZE, verbose=verbose, 
#                                          constraint_rad=0.1, num_iters=num_iters,
#                                  save_to_dir='Expirements/OptimizationPaper/' + experiment_dir + 
#                                              '/FreeformFurrie' + name + '/' + str(i), 
#                                  lr=0.005, adjust_lr_every=20, axis=a, regularization=1)

        # FreeForm
#         optimize_shape_as_pierre(model, MESH_TO_OPTIMIZE, verbose=verbose, constraint_rad=0.1, num_iters=num_iters,
#                                  save_to_dir='Expirements/OptimizationPaper/' + experiment_dir + 
#                                              '/Freeform' + name + '/' + str(i) + 'minus', 
#                                  lr=0.1, adjust_lr_every=20, axis=a, regularization=0)

#         # Scaling
#         optimize_shape_by_scaling(model, MESH_TO_OPTIMIZE, verbose=1, constraint_rad=0.1, num_iters=num_iters,
#                                  save_to_dir='Expirements/OptimizationPaper/' + experiment_dir + 
#                                              '/Scaling' + name + '/' + str(i), 
#                                  lr=0.5, adjust_lr_every=10, axis=a)

        # Vertex
        optimize_shape_by_vertex(model, MESH_TO_OPTIMIZE, verbose=1, constraint_rad=0.1, num_iters=num_iters,
                                 save_to_dir='Expirements/OptimizationPaper/' + experiment_dir + 
                                             '/Vertex' + name + '/' + str(i), 
                                 lr=0.2, adjust_lr_every=20, axis=a)

Starting shape  535
Iter  0 Loss:  -0.05752294  LR:  0.2
Iter  1 Loss:  -0.06605638  LR:  0.2
Iter  2 Loss:  -0.07459407  LR:  0.2
Iter  3 Loss:  -0.08316308  LR:  0.2
Iter  4 Loss:  -0.0918005  LR:  0.2
Iter  5 Loss:  -0.10054854  LR:  0.2
Iter  6 Loss:  -0.10945869  LR:  0.2
Iter  7 Loss:  -0.11860336  LR:  0.2
Iter  8 Loss:  -0.12808554  LR:  0.2
Iter  9 Loss:  -0.13805544  LR:  0.2
Iter  10 Loss:  -0.14875007  LR:  0.2
Iter  11 Loss:  -0.1605454  LR:  0.2
Iter  12 Loss:  -0.17405614  LR:  0.2
Iter  13 Loss:  -0.19030702  LR:  0.2
Iter  14 Loss:  -0.21103828  LR:  0.2
Iter  15 Loss:  -0.23926103  LR:  0.2
Iter  16 Loss:  -0.28027478  LR:  0.2
Iter  17 Loss:  -0.3435629  LR:  0.2
Iter  18 Loss:  -0.44643754  LR:  0.2
Iter  19 Loss:  -0.620728  LR:  0.2
Iter  20 Loss:  -0.9251458  LR:  0.1
Iter  21 Loss:  -1.176847  LR:  0.1
Iter  22 Loss:  -1.5219252  LR:  0.1
Iter  23 Loss:  -2.0004792  LR:  0.1
Iter  24 Loss:  -2.6652339  LR:  0.1
Iter  25 Loss:  -3.5955687  LR:  0.1
Iter  26 Loss:

KeyboardInterrupt: 

# Run Optimizations for the visualization

In [ ]:
experiment_dir = 'TruckVisualizations'
LATENT_TO_OPTIMIZE = latent_vectors[32]
LATENT_KD_TREE = KDTree(np.array([lv.cpu().detach().numpy()[0] for lv in latent_vectors]))

In [ ]:
def calculate_loss(mesh, local_preds, axis=0, constraint_rad=0.1):
    loss =  (1 - axis) * compute_lift_faces_diff(mesh, local_preds, axis=0) + \
                  axis * compute_lift_faces_diff(mesh, local_preds, axis=1)
    
    loss += boundsLoss(mesh.x, box=[(-0.6, 0.6, 0)])
    loss += innerBoundsLoss(mesh.x, r=constraint_rad**2, center=(-0.05, 0.05, 0))  \
          + innerBoundsLoss(mesh.x, r=(constraint_rad / 2)**2, center=(0.3, 0, 0))
    return loss

make_full_transformation(LATENT_TO_OPTIMIZE.detach(), LATENT_TO_OPTIMIZE.clone(), 
                                    'Expirements/OptimizationPaper/' + experiment_dir + '/32/cabin05', decoder, model,
                         alpha_penalty=0.2, axis=0,
                         constraint_rad=0.05,
                         num_iters=30,
                         adjust_lr_every=20,
                         decreased_by=1.1, 
                         lr=0.2,
                         verbose=None,
                         N=256,
                         num_neignours_constr=10,
                         reindex_latent_each=10000,
                         punch_lr_at_reindex_by=1,
                         reindex_num_iterations=500,
                         reindex_num_samples=100)

In [ ]:
def calculate_loss(mesh, local_preds, axis=0, constraint_rad=0.1):
    loss =  (1 - axis) * compute_lift_faces_diff(mesh, local_preds, axis=0) + \
                  axis * compute_lift_faces_diff(mesh, local_preds, axis=1)
    
    loss += boundsLoss(mesh.x, box=[(-0.6, 0.6, 0)])
    loss += innerBoundsLoss(mesh.x, r=constraint_rad**2, center=(-0.15, 0.05, 0))  \
          + innerBoundsLoss(mesh.x, r=(constraint_rad / 2)**2, center=(0.3, 0, 0))
    return loss

make_full_transformation(LATENT_TO_OPTIMIZE.detach(), LATENT_TO_OPTIMIZE.clone(), 
                                    'Expirements/OptimizationPaper/' + experiment_dir + '/32/cabinBack15', decoder, model,
                         alpha_penalty=0.2, axis=0,
                         constraint_rad=0.2,
                         num_iters=30,
                         adjust_lr_every=20,
                         decreased_by=1.1, 
                         lr=0.2,
                         verbose=1,
                         N=256,
                         num_neignours_constr=10,
                         reindex_latent_each=10000,
                         punch_lr_at_reindex_by=1,
                         reindex_num_iterations=500,
                         reindex_num_samples=100)

In [ ]:
def calculate_loss(mesh, local_preds, axis=0, constraint_rad=0.1):
    loss =  (1 - axis) * compute_lift_faces_diff(mesh, local_preds, axis=0) + \
                  axis * compute_lift_faces_diff(mesh, local_preds, axis=1)
    
    loss += boundsLoss(mesh.x, box=[(-0.6, 0.6, 0)])
    loss += innerBoundsLoss(mesh.x, r=constraint_rad**2, center=(-0.15, 0.05, 0))  \
          + innerBoundsLoss(mesh.x, r=(constraint_rad / 2)**2, center=(0.3, 0, 0))
    return loss

make_full_transformation(LATENT_TO_OPTIMIZE.detach(), LATENT_TO_OPTIMIZE.clone(), 
                                    'Expirements/OptimizationPaper/' + experiment_dir + '/32/cabinFront15', decoder, model,
                         alpha_penalty=0.2, axis=0,
                         constraint_rad=0.1,
                         num_iters=30,
                         adjust_lr_every=20,
                         decreased_by=1.1, 
                         lr=0.2,
                         verbose=1,
                         N=256,
                         num_neignours_constr=10,
                         reindex_latent_each=10000,
                         punch_lr_at_reindex_by=1,
                         reindex_num_iterations=500,
                         reindex_num_samples=100)

In [ ]:
def calculate_loss(mesh, local_preds, axis=0, constraint_rad=0.1):
    loss =  (1 - axis) * compute_lift_faces_diff(mesh, local_preds, axis=0) + \
                  axis * compute_lift_faces_diff(mesh, local_preds, axis=1)
    
#     loss += boundsLoss(mesh.x, box=[(-0.6, 0.6, 0)])
#     loss += innerBoundsLoss(mesh.x, r=constraint_rad**2, center=(0.05, 0.05, 0))  \
#           + innerBoundsLoss(mesh.x, r=(constraint_rad / 2)**2, center=(0.3, 0, 0))
    return loss

make_full_transformation(LATENT_TO_OPTIMIZE.detach(), LATENT_TO_OPTIMIZE.clone(), 
                                    'Expirements/OptimizationPaper/' + experiment_dir + '/32/axis0.2', decoder, model,
                         alpha_penalty=0.3, axis=0.2,
                         constraint_rad=0.1,
                         num_iters=100,
                         adjust_lr_every=20,
                         decreased_by=1.1, 
                         lr=0.2,
                         verbose=None,
                         N=256,
                         num_neignours_constr=10,
                         reindex_latent_each=10000,
                         punch_lr_at_reindex_by=1,
                         reindex_num_iterations=500,
                         reindex_num_samples=100)

In [ ]:
def calculate_loss(mesh, local_preds, axis=0, constraint_rad=0.1):
    loss =  (1 - axis) * compute_lift_faces_diff(mesh, local_preds, axis=0) + \
                  axis * compute_lift_faces_diff(mesh, local_preds, axis=1)
    
#     loss += boundsLoss(mesh.x, box=[(-0.6, 0.6, 0)])
#     loss += innerBoundsLoss(mesh.x, r=constraint_rad**2, center=(0.05, 0.05, 0))  \
#           + innerBoundsLoss(mesh.x, r=(constraint_rad / 2)**2, center=(0.3, 0, 0))
    return loss

make_full_transformation(LATENT_TO_OPTIMIZE.detach(), LATENT_TO_OPTIMIZE.clone(), 
                                    'Expirements/OptimizationPaper/' + experiment_dir + '/32/axis0.5', decoder, model,
                         alpha_penalty=0.3, axis=0.2,
                         constraint_rad=0.1,
                         num_iters=100,
                         adjust_lr_every=20,
                         decreased_by=1.1, 
                         lr=0.2,
                         verbose=None,
                         N=256,
                         num_neignours_constr=10,
                         reindex_latent_each=10000,
                         punch_lr_at_reindex_by=1,
                         reindex_num_iterations=500,
                         reindex_num_samples=100)

In [ ]:
def calculate_loss(mesh, local_preds, axis=0, constraint_rad=0.1):
    loss =  (1 - axis) * compute_lift_faces_diff(mesh, local_preds, axis=0) + \
                  axis * compute_lift_faces_diff(mesh, local_preds, axis=1)
    
#     loss += boundsLoss(mesh.x, box=[(-0.6, 0.6, 0)])
#     loss += innerBoundsLoss(mesh.x, r=constraint_rad**2, center=(0.05, 0.05, 0))  \
#           + innerBoundsLoss(mesh.x, r=(constraint_rad / 2)**2, center=(0.3, 0, 0))
    return loss

make_full_transformation(LATENT_TO_OPTIMIZE.detach(), LATENT_TO_OPTIMIZE.clone(), 
                                    'Expirements/OptimizationPaper/' + experiment_dir + '/32/axis0.8', decoder, model,
                         alpha_penalty=0.3, axis=0.8,
                         constraint_rad=0.1,
                         num_iters=100,
                         adjust_lr_every=20,
                         decreased_by=1.1, 
                         lr=0.2,
                         verbose=None,
                         N=256,
                         num_neignours_constr=10,
                         reindex_latent_each=10000,
                         punch_lr_at_reindex_by=1,
                         reindex_num_iterations=500,
                         reindex_num_samples=100)

In [ ]:
def calculate_loss(mesh, local_preds, axis=0, constraint_rad=0.1):
    loss =  (1 - axis) * compute_lift_faces_diff(mesh, local_preds, axis=0) + \
                  axis * compute_lift_faces_diff(mesh, local_preds, axis=1)
    
    loss += boundsLoss(mesh.x, box=[(-0.6, 0.6, 0)])
    loss += innerBoundsLoss(mesh.x, r=constraint_rad**2, center=(-0.15, 0.05, 0))  \
          + innerBoundsLoss(mesh.x, r=(constraint_rad / 2)**2, center=(0.3, 0, 0))
    return loss

make_full_transformation(LATENT_TO_OPTIMIZE.detach(), LATENT_TO_OPTIMIZE.clone(), 
                                    'Expirements/OptimizationPaper/' + experiment_dir + '/32/cabinBackSmall15', decoder, model,
                         alpha_penalty=0.2, axis=0,
                         constraint_rad=0.05,
                         num_iters=30,
                         adjust_lr_every=20,
                         decreased_by=1.1, 
                         lr=0.2,
                         verbose=None,
                         N=256,
                         num_neignours_constr=10,
                         reindex_latent_each=10000,
                         punch_lr_at_reindex_by=1,
                         reindex_num_iterations=500,
                         reindex_num_samples=100)

# Check Simulation

In [ ]:
from numpy import genfromtxt
my_data = genfromtxt('../Expirements/SavedTransforms/DeepSDF-CFD8-m3-holes/report/00000/output/cloud_p_k_omega_nut.csv', 
                     delimiter=',', skip_header=1)

In [ ]:
mesh = trimesh.load("../Data/cars_remeshed_dsdf/inputs/0_input.stl")
norm = mpl.colors.Normalize(vmin= -2, vmax=2)
cmap = cm.hot
m = cm.ScalarMappable(norm=norm, cmap=cmap)

In [ ]:
fld_tree = KDTree(my_data[:, :3])
distances, indeces = fld_tree.query(mesh.vertices, k=1)

interpolations = my_data[:, 0][indeces].squeeze()
mesh = trimesh.Trimesh(vertices=mesh.vertices, faces=mesh.faces, 
                           vertex_colors=list(map(lambda c: m.to_rgba(c),  interpolations)))

In [ ]:
mesh = trimesh.load("../Data/cars_refined/simulated/stl/0005.stl")
mesh.show()

#### From processed

In [ ]:
di = make_data_instance_from_stl("/cvlabdata2/home/artem/Data/check_simulations/Processed/fld/0002_0005.fld")
di.to("cuda:0")

In [ ]:
mesh = get_trimesh_from_torch_geo_with_colors(di, di.y)
lift = compute_lift_faces(di, di.y)
print('Lift: ', lift)

In [ ]:
plt.hist(di.y[:, 0].detach().cpu().numpy(), bins=100)
plt.show()

In [ ]:
mesh.show()

In [ ]:
npar = np.genfromtxt("/cvlabdata2/home/artem/Data/check_simulations/Processed/fld/0002_0005.fld", delimiter=',', skip_header=1)
plt.hist(npar[:, 3], bins=100, alpha = 0.5)
npar = np.genfromtxt("/cvlabdata2/home/artem/Data/check_simulations/Processed/fld/0000_0005.fld", delimiter=',', skip_header=1)
plt.hist(npar[:, 3], bins=100, alpha = 0.5)
npar = np.genfromtxt("/cvlabdata2/home/artem/Data/check_simulations/Processed/fld/0001_0005.fld", delimiter=',', skip_header=1)
plt.hist(npar[:, 3], bins=100, alpha = 0.5)
plt.show()

In [ ]:
npar = np.genfromtxt("/cvlabdata2/home/artem/Data/cars_refined/simulated/fld/0002_0005.fld", delimiter=',', skip_header=1)
plt.hist(npar[:, 3], bins=100, alpha = 0.5)
npar = np.genfromtxt("/cvlabdata2/home/artem/Data/cars_refined/simulated/fld/0003_0005.fld", delimiter=',', skip_header=1)
plt.hist(npar[:, 3], bins=100, alpha = 0.5)
npar = np.genfromtxt("/cvlabdata2/home/artem/Data/cars_refined/simulated/fld/0007_0005.fld", delimiter=',', skip_header=1)
plt.hist(npar[:, 3], bins=100, alpha = 0.5)
plt.show()

In [ ]:
npar = np.genfromtxt("/cvlabdata2/home/artem/Data/check_simulations/ListenerCheck/processed/fld/0000_0005.fld", delimiter=',', skip_header=1)
plt.hist(npar[:, 3], bins=100, alpha = 0.5)
npar = np.genfromtxt("/cvlabdata2/home/artem/Data/check_simulations/ListenerCheck/processed/fld/0001_0005.fld", delimiter=',', skip_header=1)
plt.hist(npar[:, 3], bins=100, alpha = 0.5)
npar = np.genfromtxt("/cvlabdata2/home/artem/Data/check_simulations/ListenerCheck/processed/fld/0002_0005.fld", delimiter=',', skip_header=1)
plt.hist(npar[:, 3], bins=100, alpha = 0.5)
plt.show()

In [ ]:
di0 = make_data_instance_from_stl("../Expirements/SimulatedResults/foam_npy/fld/0000_0005.fld")
di1 = make_data_instance_from_stl("../Expirements/SimulatedResults/foam_npy/fld/0001_0005.fld")

In [ ]:
plt.hist(di0.y[:, 0], bins=100)
plt.show()

In [ ]:
plt.hist(di1.y[:, 0], bins=100)
plt.show()

In [ ]:
mesh = trimesh.load('/cvlabdata2/home/artem/Data/cars_refined/simulated/stl/0005.stl')

In [ ]:
mesh.show()

In [ ]:
path = '../Expirements/SavedTransforms/DeepSDF-CFD8-m3-holes/meshes/00000.ply'
tri_mesh = trimesh.load(path)
points = torch.tensor(tri_mesh.vertices, dtype=torch.float).to("cuda:0")

In [ ]:
innerBoundsLoss(points, r=0.12**2, center=(0.55, 0, 0))

In [ ]:
from numpy import genfromtxt
bvecs = genfromtxt('examples/cars/vine_latentVecs_tll_0.1_1.csv', delimiter=' ', skip_header=1)

In [ ]:
bvecs = [torch.FloatTensor(v).to('cuda:0') for v in bvecs]

In [ ]:
ply_mesh = create_mesh( decoder,
                        latent_vectors[0],
                        N=128,
                        max_batch=int(2 ** 18),
                        offset=None,
                        scale=None)

points = torch.cuda.FloatTensor(
    np.array([ply_mesh['vertex']['x'], ply_mesh['vertex']['y'], ply_mesh['vertex']['z']]).transpose()
)

transformed_points = make_mesh_from_points(points, ply_mesh)

In [ ]:
pred = model(transformed_points)

In [ ]:
mesh = get_trimesh_from_torch_geo_with_colors(transformed_points, pred.cpu().detach())

In [ ]:
# mesh.show()

In [ ]:
n = mesh.export('examples/cars/BeautifiedMeshes/o0.ply')

In [ ]:
get_latent_from_mesh_cpu()

In [ ]:
edges = trimesh.geometry.faces_to_edges(ply_mesh['face']['vertex_indices'])
edge_attr = torch.stack([points[a] - points[b] for a, b in edges])

In [ ]:
print(edge_attr[:10])

In [ ]:
cloud = PyntCloud(pd.DataFrame(points.cpu().detach().numpy(), columns=['x', 'y', 'z']))
cloud.plot(background='white', initial_point_size=0.003)

In [ ]:
cloud = PyntCloud(pd.DataFrame(points_transformed.cpu().detach().numpy(), columns=['x', 'y', 'z']))
cloud.plot(background='white', initial_point_size=0.003)

In [ ]:
di = make_data_instance_from_stl('/cvlabdata2/home/artem/Data/cars_refined/simulated/fld/0007_0005.fld')

In [ ]:
cm = get_trimesh_from_torch_geo_with_colors(di, di.y)

In [ ]:
preds = model()

In [ ]:
cm.show()

In [ ]:
transformed_points

In [ ]:
cloud = PyntCloud(pd.DataFrame(di.pos.cpu().detach().numpy(), columns=['x', 'y', 'z']))
cloud.plot(background='white', initial_point_size=0.003)

In [ ]:
cloud = PyntCloud(pd.DataFrame(transformed_points.pos.cpu().detach().numpy(), columns=['x', 'y', 'z']))
cloud.plot(background='white', initial_point_size=0.003)

In [ ]:
with torch.no_grad():
    ply_mesh = create_mesh( decoder,
                            latent,
                            N=128,
                            max_batch=int(2 ** 18),
                            offset=None,
                            scale=None)

points = torch.cuda.FloatTensor(np.hstack(( ply_mesh['vertex']['x'][:, None], 
                                            ply_mesh['vertex']['y'][:, None], 
                                            ply_mesh['vertex']['z'][:, None])))

mesh = trimesh.Trimesh(vertices=points.cpu().detach(), faces=ply_mesh['face']['vertex_indices'])
edge_attr = [mesh.vertices[a] - mesh.vertices[b] for a, b in mesh.edges]

data = torch_geometric.data.Data(x  = points, 
                                 pos= torch.tensor(mesh.vertices, dtype=torch.float).to('cuda:0'), 
                                 faces = torch.tensor(mesh.faces, dtype=torch.long).to('cuda:0'),
                                 edge_attr = torch.tensor(edge_attr, dtype=torch.float).to('cuda:0'),
                                 edge_index= torch.tensor(mesh.edges, dtype=torch.long).t().contiguous().to('cuda:0'))

In [ ]:
data_instance= make_data_instance_from_stl(
                    '/cvlabsrc1/cvlab/dataset_shapenet/code/foam_npy/fld/0100_0005.fld', data_step=1)
mesh = trimesh.Trimesh(vertices=data_instance.pos, faces=data_instance.faces)
a = mesh.export('../Expirements/data/original_mesh.ply')

# Expirements with PreprocessMesh

In [ ]:
mesh.show()

In [ ]:
latent = get_latent_from_mesh_cpu(decoder, latent_size, mesh, num_iterations=300, num_samples=200000)

In [ ]:
latent_gpu = get_latent_from_mesh(decoder, latent_size, mesh_path='../Expirements/data/original_mesh.ply', 
                         num_iterations=300, num_samples=200000)

In [ ]:
gpu_points = deep_sdf.data.read_sdf_samples_into_ram('../Expirements/data/original_SDF.npz')
gpu_points[1][:, 3].max()

In [ ]:
cloud = PyntCloud(pd.DataFrame(np.array(gpu_points[0][:, :3]), columns=['x', 'y', 'z']))
cloud.plot(background='white', initial_point_size=0.03, elev=-45, azim=45)

In [ ]:
points, sdf = sample_sdf_near_surface(mesh)
sdfs = np.hstack((points, sdf[:, None]))
data_sdf = [torch.from_numpy(sdfs[sdfs[:, 3] > 0, :]), 
            torch.from_numpy(sdfs[sdfs[:, 3] < 0, :])]

In [ ]:
print(data_sdf[0].shape)
print(gpu_points[0].shape)

In [ ]:
cloud = PyntCloud(pd.DataFrame(np.array(data_sdf[0][:, :3]), columns=['x', 'y', 'z']))
cloud.plot(background='white', initial_point_size=0.03, elev=-45, azim=45)

In [ ]:
plot_mesh_from_vector(decoder, latent, N=128)

In [ ]:
plydata = create_mesh(decoder, latent_gpu)

In [ ]:
plydata.write("../Expirements/mesh.ply")